In [ ]:
!pip install google-generativeai

  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached google_api_core-2.24.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_api_python_client-2.169.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached google_auth-2.40.1-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached protobuf-5.29.4-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached grpcio_status-1.71.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
Using cached google_generativeai

In [6]:
!pip install PyMuPDF

  Using cached pymupdf-1.25.5-cp39-abi3-win_amd64.whl.metadata (3.4 kB)
Using cached pymupdf-1.25.5-cp39-abi3-win_amd64.whl (16.6 MB)


In [8]:
!pip install python-dotenv

In [10]:
!pip install sentence-transformers faiss-cpu

   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ----------- ---------------------------- 2.9/10.4 MB 14.0 MB/s eta 0:00:01
   ---------------------- ----------------- 5.8/10.4 MB 13.6 MB/s eta 0:00:01
   ------------------------------- -------- 8.1/10.4 MB 12.6 MB/s eta 0:00:01
   ---------------------------------------- 10.4/10.4 MB 13.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 15.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   ------ --------------------------------- 2.6/15.0 MB 13.7 MB/s eta 0:00:01
   -------------- ------------------------- 5.5/15.0 MB 14.0 MB/s eta 0:00:01
   --------------------- ------------------ 8.1/15.0 MB 13.6 MB/s eta 0:00:01
   ---------------------------- ----------- 10.7/15.0 MB 13.4 MB/s eta 0:00:01
   ----------------------------------- ---- 13.4/15.0 MB 13.5 MB/s eta 0:00:01
   ------

In [26]:
!pip install streamlit

   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------- ----------------------------- 2.6/9.9 MB 15.1 MB/s eta 0:00:01
   ---------------------- ----------------- 5.5/9.9 MB 14.0 MB/s eta 0:00:01
   ------------------------------- -------- 7.9/9.9 MB 12.8 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 13.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   --------------------------------------- 731.2/731.2 kB 10.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   --------- ------------------------------ 2.9/11.6 MB 15.2 MB/s eta 0:00:01
   ------------------ --------------------- 5.5/11.6 MB 14.0 MB/s eta 0:00:01
   ---------------------------- ----------- 8.1/11.6 MB 13.6 MB/s eta 0:00:01
   ------------------------------------- -- 11.0/11.6 MB 13.5 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 13.2 MB/s eta 0:00:00
   -------

In [12]:
import google.generativeai as genai
import fitz
from dotenv import load_dotenv
import os
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

In [14]:
# Ladda API-nyckel från .env
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

# Konfigurera Google API
genai.configure(api_key=api_key)

# Skapa en modellinstans
model = genai.GenerativeModel("gemini-1.5-flash")  # eller "gemini-1.0-pro" beroende på API-åtkomst

# Skicka fråga
response = model.generate_content("Explain how AI works in a few words")

# Skriv ut svar
print(response.text)


Learning patterns from data to make predictions.



In [ ]:
# Funktion för att hämta information från lagarna jag valt

def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text


pdf_text1 = extract_text_from_pdf("pdfs/AnstallningsskyddLag.pdf")
pdf_text2 = extract_text_from_pdf("pdfs/Arbetstidslag.pdf")
pdf_text3 = extract_text_from_pdf("pdfs/MedbestammandeLag.pdf")
pdf_text4 = extract_text_from_pdf("pdfs/Offentliganstallning.pdf")
pdf_text5 = extract_text_from_pdf("pdfs/RattegangArbetstvist.pdf")
pdf_text6 = extract_text_from_pdf("pdfs/SemesterLag.pdf")

all_texts = "\n\n".join([
    pdf_text1,
    pdf_text2,
    pdf_text3,
    pdf_text4,
    pdf_text5,
    pdf_text6
])

In [ ]:
# Funktion för att splitta texten till olika "chunks"
def split_text_into_chunks(text, max_length=1000):
    chunks = []
    current_chunk = ""

    for paragraph in text.split("\n"):
        paragraph = paragraph.strip()
        if not paragraph:
            continue

        if len(current_chunk) + len(paragraph) < max_length:
            current_chunk += paragraph + "\n"
        else:
            chunks.append(current_chunk.strip())
            current_chunk = paragraph + "\n"

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

In [ ]:
# Funktion för att inkludera källan till de olika chunks vi har skapat
def make_chunks_with_source(texts, sources, max_length=1000):
    result = []
    for text, source in zip(texts, sources):
        split = split_text_into_chunks(text, max_length)
        for chunk in split:
            result.append({
                "text": chunk,
                "source": source
            })
    return result

# Exempel:
sources = [
    "Anställningsskydd", "Arbetstidslag", "Medbestämmande",
    "Offentliganställning", "Arbetstvist", "Semesterlag"
]
all_chunks = make_chunks_with_source(
    [pdf_text1, pdf_text2, pdf_text3, pdf_text4, pdf_text5, pdf_text6],
    sources
)

In [18]:
chunks = split_text_into_chunks(all_texts, max_length=1000)
print(f"Antal chunkar: {len(chunks)}")
print(chunks[0])  # Förhandsgranska första chunken

Antal chunkar: 229
SFS 1982:80
Lag (1982:80) om anställningsskydd
Om författningen
Prop. 1981/82:71, AU11 – Ang. ändringslagen 1996:1424 prop. 1996/97:16 och bet. 1996/97:AU4.
Utfärdad den 24 februari 1982.
Karnov kommentarerna uppdaterade:
Kommentaren uppdaterad per 1 januari 2025
Kommentaren genomgången per 1 juli 2024
Inledande bestämmelser
1 §
Denna lag gäller arbetstagare i allmän eller enskild tjänst.
Från lagens tillämpning undantas dock
1. arbetstagare som är anställda för arbete i arbetsgivarens hushåll, och
2. arbetstagare som är anställda i gymnasial lärlingsanställning.
Bestämmelserna i 4–6 b, 6 f, 6 g, 7–20, 22–37, 39 och 40 §§ gäller inte för
1. arbetstagare som med hänsyn till arbetsuppgifter och anställningsvillkor får anses ha företagsledande eller därmed
jämförlig ställning,
2. arbetstagare som tillhör arbetsgivarens familj, och
3. arbetstagare som är anställda med särskilt anställningsstöd, i skyddat arbete eller med lönebidrag för utveckling i
anställning.
2 §


In [19]:
# Ladda modellen
model = SentenceTransformer("all-MiniLM-L6-v2")

# Antag att du har en lista: chunks = [chunk1, chunk2, chunk3, ...]
embeddings = model.encode(chunks, convert_to_numpy=True)

print(f"Skapade {len(embeddings)} embeddings.")

c:\Users\46733\pyhtongithub\kurs_pystat\ds24_deep_learning\Kunskapskontroll 2\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\46733\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 

Skapade 229 embeddings.


In [20]:
# Skapa index med rätt dimension
embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)

# Lägg till embeddings i indexet
index.add(embeddings)

print(f"FAISS-index innehåller {index.ntotal} chunkar.")

FAISS-index innehåller 229 chunkar.


In [ ]:

chunk_lookup = chunks  # index i FAISS matchar index här

In [22]:
query = "Vad gäller vid uppsägning på grund av arbetsbrist?"
query_embedding = model.encode([query])

# Sök topp 3 mest relevanta chunkar
D, I = index.search(np.array(query_embedding), k=3)

for idx in I[0]:
    print("---")
    print(chunk_lookup[idx])

---
Om två eller flera arbetsgivar- eller arbetstagarorganisationer slås samman, skall kollektivavtal som gäller för
organisation som upplöses, gälla för den sammanslagna organisationen som om avtalet hade slutits av denna.
29 §
Är kollektivavtal på den ena sidan eller på båda sidor slutet av flera parter och kräves uppsägning för att avtalet skall
upphöra att gälla, får part för egen del säga upp avtalet hos en eller flera parter på andra sidan. Har sådan uppsägning
skett till viss tid, får annan part säga upp avtalet till samma tidpunkt. Dennes uppsägning skall dock göras inom tre
veckor efter det att uppsägning annars skulle ha skett eller, om den avtalade uppsägningstiden understiger sex veckor,
inom hälften av uppsägningstiden.
30 §
Uppsägning av kollektivavtal skall ske skriftligen.
Har meddelande om uppsägning avsänts under motpartens senast kända adress så tidigt att det hade bort komma
---
eller återintagning efter permittering gäller 11–14 §§ lagen (1976:580) om medbestämmand

In [23]:
def retrieve_relevant_chunks(query, model, index, chunk_lookup, k=3):
    query_embedding = model.encode([query])
    D, I = index.search(query_embedding, k)
    return [chunk_lookup[i] for i in I[0]]

In [ ]:
#Skapar en prompt där de mest relevanta lagtext-chunkarna samt frågan skickas till Gemini.
def generate_gemini_answer(question, context_chunks):
    context = "\n\n".join(context_chunks)

    prompt = f"""
Du är en juridisk assistent med fokus på svensk arbetsrätt. Använd följande utdrag ur lagtexten för att svara på frågan.

LAGTEXT:
{context}

FRÅGA:
{question}

SVAR:"""

    model = genai.GenerativeModel("gemini-1.5-flash")  # eller annan modell du har tillgång till
    response = model.generate_content(prompt)
    return response.text

In [ ]:
# Användarens fråga
user_question = "Vad gäller vid uppsägning på grund av arbetsbrist?"

# Hämta relevant kontext
relevant_chunks = retrieve_relevant_chunks(user_question, model, index, chunks)

# Generera svar med Gemini
answer = generate_gemini_answer(user_question, relevant_chunks)

print("🔍 Fråga:", user_question)
print("🧾 Svar:", answer)

🔍 Fråga: Vad gäller vid uppsägning på grund av arbetsbrist?
🧾 Svar: Den givna lagtexten innehåller ingen information om uppsägning på grund av arbetsbrist.  Lagtexten behandlar istället regler kring sammanslagning av organisationer, uppsägning av kollektivavtal och uppsägning/avskedande av enskilda arbetstagare på grund av personliga omständigheter.  För att besvara frågan om uppsägning på grund av arbetsbrist krävs tillgång till andra delar av svensk arbetsrätt, exempelvis lagen om anställningsskydd (LAS).



Diskussion

I slutet av din kod ska du redogöra för hur din modell hade kunnat användas i verkligheten och vilka potentiella utmaningar och möjligheter (t.ex. affärsmässiga, etiska och andra perspektiv du finner relevanta) som finns. Du kan skriva detta som kommentarer i koden.

Min chattbot hade kunnat användas inom juridiska situationer, specifikt för en advokatsbyrå eller för HR inom näringslivet. Utmaningarna inom juridik och för en advokatsbyrå skulle kunna vara att hålla den uppdaterad kring alla praxis och nya dokument som hela tiden sker inom Arbetsrätt. Jag tror även att den här typen av automatisering börjar bli mer vanligt så man hade behövt ha koden/appen väldigt konkurrenskraftig. T.ex. med ett väldigt välarbetat UI och bra relevanta funktioner som
kan hjälpa till att göra day-to-day arbetet ännu enklare för användaren.

Möjligheter men även etiska perspektiv handlar om automatiseringen som man gör med hjälp av denna chattbot. Mindre behov av personal/ juridik assistenter etc. Men möjligheten är den samma, sparar in på kostnader och gör företagen mer lönsamma.

Diskussion kring att använda ett verktyg/modell för att ta fram lagar inom ett känsligt område som juridik, specifikt i kriminella rättegånger (Arbetsrätt kanske inte lika känsligt) som avgör framtiden för en individ bör absolut tas i åtanke. Men på samma sätt som en modell kan göra fel kan faktiskt även en människa fela. Vad en juridikbot såsom denna bör ses som är precis vad den är, ett verktyg, ett hjälpmedel för människor att utföra sina jobb mer effektivit och enkelt. Det ska inte ses som ett beslutfattande utan mer som en hjälpreda.


Självutvärdering
1. Vad har varit roligast i kunskapskontrollen?

 Skapandet av chattbotten har varit kul och spännande. Det känns verkligen relevant och det är lätt att se hur vi kan använda detta i verkligheten och i situationer i näringslivet.

2. Vilket betyg anser du att du ska ha och varför?

Jag anser att jag förtjänar G eftersom jag har utfört alla grundläggande moment i kursen.

3. Vad har varit mest utmanande i arbetet och hur har du hanterat det?

Lite svårt att förhålla sig till studieplanen när den är såppas flexibel som den har varit i denna kurs. Med det syftar jag på de frivilliga texterna som vi kunde läsa men ej var obligatoriska t.ex. Jag personligen uppskattar mer praktiska uppgifter att jobba med än teoritisk litteratur och det har det funnits en del i denna kurs. 
